# Import related functions

In [1]:
import sys
import cobra
import pandas as pd
import re
sys.path.append(r'./script/')
from ECMpy_function import *
from AutoPACMEN_function import *

# Input and output files

In [2]:
# Initial parameter
obj='CG_biomass_cgl_ATCC13032'# CG_biomass_cgl_ATCC13032 EX_lys_L_e
fluxes_outfile = './analysis/ECMpy_solution_%s_pfba.csv'%obj
use_substrate='EX_glc_e'
concentration=10
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 0.5
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
enz_ratio=0.1

# Input files
autopacmen_folder = "./iCW773_get_data/"
gap_fill= 'mean'#'mean'#'median'
project_name = "iCW773_%s"%gap_fill
organism = "Corynebacterium glutamicum"
create_file(autopacmen_folder)
reaction_gene_path = "./data/reaction_gene_subunit_num.json"
#reaction_gene_path='none'
protein_kcat_database_path = "none"
bigg_metabolites_file = "./data/bigg_models_metabolites.txt"#date:20211008 http://bigg.ucsd.edu/static/namespace/bigg_models_metabolites.txt
# brenda_download.txt exceeds GitHub's file size limit of 100.00 MB, so we not upload to github
brenda_textfile_path = "./data/brenda_download.txt"#date:20211008 https://www.brenda-enzymes.org/brenda_download/file_download.php
sbml_path = "./model/iCW773_uniprot_modification_del.xml"
reaction_kcat_MW_file="./data/reaction_kcat_MW.csv"
gene_abundance_file1='./data/2020_GSM4849478_WT_normalized_counts_2020.tsv'
gene_abundance_file2='./data/2019_GSM3421586_wt.tsv'
gene_abundance_file3='./data/2019_GSM3421586_wt2.tsv'
file_list=[gene_abundance_file1,gene_abundance_file2,gene_abundance_file3]
EC_max_file='./data/EC_kcat_max.json'

# Output files
brenda_json_path = "%skcat_database_brenda.json"%autopacmen_folder
brenda_output_json_path = "%skcat_database_brenda_for_model.json"%autopacmen_folder
bigg_id_name_mapping_path = "%sbigg_id_name_mapping.json"%autopacmen_folder
sabio_rk_output_path = "%skcat_database_sabio_rk.json"%autopacmen_folder
combined_output_path = "%skcat_database_combined.json"%autopacmen_folder
reaction_kcat_MW_file="%s/reaction_kcat_MW_auto.csv"%autopacmen_folder
uniprot_data_file='%s/%s_protein_id_mass_mapping.json'%(autopacmen_folder,project_name)

modelgene2uniprot_file='./analysis/modelgene2uniprot.txt'
paxdbgene2uniprot_file='./analysis/paxdbgene2uniprot.txt'
ecModel_output_file="./model/iCW773_irr_enz_constraint.json" # iCW773_irr_enz_constraint_autoECMpy.json iCW773_irr_enz_constraint.json
json_output_file = './model/iCW773_irr_enz_constraint_adj.json'
reaction_kcat_MW_outfile = './analysis/reaction_change_by_enzuse.csv'

fluxes_final_outfile = './analysis/ECMpy_solution_%s_pfba_PDH.csv'%obj
ecModel_final_output_file="./model/iCW773_irr_enz_constraint_adj_PDH.json"
reaction_kcat_MW_final_file='./analysis/reaction_change_by_enzuse_PDH.csv'

path exists


# Get reaction kcat_mw using AutoPacmen

## https://github.com/klamt-lab/autopacmen

In [3]:
import datetime 
starttime=datetime.datetime.now()
# Step 1: get bigg metbolite
create_file(autopacmen_folder)
print("Starting to deal BIGG metabolites text file...")
parse_bigg_metabolites_file(bigg_metabolites_file, autopacmen_folder)
print("BIGG metabolites text file done!")
print()

# Step 2: BRENDA kcat
print("Starting to deal BRENDA textfile...")
parse_brenda_textfile(brenda_textfile_path, autopacmen_folder, brenda_json_path)
print("BRENDA textfile done!")
print()

# Step 3: Select Brenda kcat for model
print("Starting to deal brenda json for model...")
parse_brenda_json_for_model(sbml_path, brenda_json_path, brenda_output_json_path)
print("BRENDA json for model done!")
print()

# Step 4: SABIO-RK kcat for model
print("Starting EC numbers kcat search in SABIO-RK...")
parse_sabio_rk_for_model_with_sbml(sbml_path, sabio_rk_output_path, bigg_id_name_mapping_path)
print("SABIO-RK done!")
print()

# Step 5: Brenda and SABIO-RK kcat combined
print("Combining kcat database...")
create_combined_kcat_database(sabio_rk_output_path, brenda_output_json_path, combined_output_path)
print("Combining kcat database done!")
print()

# Step 6: kcat assignment for model
print("Starting to assign kcat for model...")
get_reactions_kcat_mapping(sbml_path, autopacmen_folder, project_name, organism, combined_output_path, protein_kcat_database_path,gap_fill)
print("kcat assignment done!")
print()

# Step 7: get mw for model gene (must be uniprot ID)
print("Starting UniProt ID<->Protein mass search using UniProt API...")
get_protein_mass_mapping_with_sbml(sbml_path, autopacmen_folder, project_name)
print("Protein ID<->Mass mapping done!")

# Step 8: get_reaction_kcat_mw for model
print("Starting to get reaction kcat_mw for model...")
get_reaction_kcat_mw(sbml_path,autopacmen_folder, project_name, reaction_gene_path, gap_fill)       
print("reaction kcat_mw done!")
endtime=datetime.datetime.now()
print(endtime-starttime)

path exists
Starting to deal BIGG metabolites text file...
BIGG metabolites text file done!

Starting to deal BRENDA textfile...
BRENDA textfile done!

Starting to deal brenda json for model...
BRENDA json for model done!

Starting EC numbers kcat search in SABIO-RK...
SABIO-RK done!

Combining kcat database...
Combining kcat database done!

Starting to assign kcat for model...
kcat assignment done!

Starting UniProt ID<->Protein mass search using UniProt API...
Protein ID<->Mass mapping done!
Starting to get reaction kcat_mw for model...
Default kcat is: 7398.8133918117555
reaction kcat_mw done!
0:12:49.022322


# Calculate f-value

In [4]:
#gene_list
model=cobra.io.read_sbml_model(sbml_path)
model_gene_list=[]
for eachg in model.genes:
    model_gene_list.append(eachg.id)
uni_model_gene_list=list(set(model_gene_list))

GENENAME_2_ACC_from_uniprot(uni_model_gene_list,modelgene2uniprot_file)

f_list=[]
for g_a_file in file_list:
    print(g_a_file)
    gene_abundance = pd.read_csv(g_a_file, index_col='gene_id',sep='\t')
    uni_paxdb_gene_list=list(set(list(gene_abundance.index)))
    GENENAME_2_ACC_from_uniprot(uni_paxdb_gene_list,paxdbgene2uniprot_file)
    f=calculate_f_special(g_a_file, modelgene2uniprot_file,paxdbgene2uniprot_file,'normalized_counts')
    f_list.append(f)
f_list

./data/2020_GSM4849478_WT_normalized_counts_2020.tsv
./data/2019_GSM3421586_wt.tsv
./data/2019_GSM3421586_wt2.tsv


[0.45311986236929197, 0.4622348377433211, 0.46008010403741123]

# Get ecModel and simulation

In [5]:
#The enzyme mass fraction 
#f_list = [0.45311986236929197,0.4622348377433211,0.4600801040374112]
f = round(np.mean(f_list), 2)
print("f is:" + str(f))
upperbound = round(ptot * f * sigma, 3)
# upperbound = 0.1
print("upperbound is:" + str(upperbound))

# reaction_kcat_MW_file="./data/reaction_kcat_MW_auto.csv"
reaction_kcat_MW_file="%s/reaction_kcat_MW.csv"%autopacmen_folder
# reaction_kcat_MW_file='./analysis/reaction_change_by_enzuse_PDH.csv'

trans_model2enz_json_model_split_isoenzyme(sbml_path, reaction_kcat_MW_file, f, ptot, sigma, lowerbound, upperbound, ecModel_output_file)
enz_model=get_enzyme_constraint_model(ecModel_output_file)

enz_model.objective=obj

#Using only the provided substrate
[ori_obj_id,ori_substrate_id_list,ori_sub_concentration,ori_ATPM]=get_model_substrate_obj(enz_model)
for eachsubid in ori_substrate_id_list:
    if re.search('_reverse',eachsubid):
        r_id_new=eachsubid.split('_reverse')[0]
        enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
        enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0)  
    else:
        r_id_new=eachsubid+'_reverse'
        enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
        enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0) 
        
enz_model.reactions.get_by_id(use_substrate).bounds = (-10, 0)
enz_model.reactions.get_by_id(use_substrate+'_reverse').bounds = (0, 0)

enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,ecModel_output_file)
print(enz_model_pfba_solution.fluxes[obj])

f is:0.46
upperbound is:0.129
0.129
Enzyme cost total is:0.129
0.1234735854383946


# Calibration kcat according Enzyme usage

In [6]:
#Initial model and result
fluxes_infile_ori = './analysis/ECMpy_solution_%s_pfba.csv'%obj
fluxes_outfile = fluxes_infile_ori
need_change_reaction_list=[]
changed_reaction_list=[]
round_num=1
reaction_kcat_mw = pd.read_csv(reaction_kcat_MW_file, index_col=0)

#Model adjustment
enz_model_obj_max=0
concentration = 5.05
# doi: 10.1016/j.ymben.2018.05.004. Epub 2018 May 9. PMID: 29753071.
while (enz_model_obj_max<0.45 or enz_model_sub>10):

 

    [enz_model,reaction_kcat_mw,need_change_reaction_list, changed_reaction_list]=change_enz_model_by_enz_usage(enz_ratio,\
                                                    ecModel_output_file,fluxes_outfile,EC_max_file,\
                                                    reaction_kcat_mw,need_change_reaction_list,changed_reaction_list,f, \
                                                    ptot, sigma, lowerbound, upperbound, json_output_file)
    #Using only the provided substrate
    [ori_obj_id,ori_substrate_id_list,ori_sub_concentration,ori_ATPM]=get_model_substrate_obj(enz_model)
    for eachsubid in ori_substrate_id_list:
        if re.search('_reverse',eachsubid):
            r_id_new=eachsubid.split('_reverse')[0]
            enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
            enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0)  
        else:
            r_id_new=eachsubid+'_reverse'
            enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
            enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0) 
    enz_model.reactions.get_by_id(use_substrate).bounds = (-1000, 0)
    enz_model.reactions.get_by_id(use_substrate+'_reverse').bounds = (0, 0)
    enz_model_obj=enz_model.slim_optimize()

    enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
    fluxes_outfile = './analysis/ECMpy_solution_adjround%s.csv'%(round_num)
    enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,json_output_file)
    
    

   # μmax condition substrate concentration
    obj='CG_biomass_cgl_ATCC13032'# CG_biomass_cgl_ATCC13032 EX_lys_L_e EX_trp_L_e
    substrate='EX_glc_e'
    enz_model.reactions.get_by_id(substrate).bounds=(-1000,0)
    enz_model.reactions.get_by_id('EX_glc_e_reverse').bounds=(0,0)
    # enz_model.reactions.get_by_id('CG_biomass_cgl_ATCC13032').bounds=(0.1,0.1)
    enz_model.objective=obj
    enz_model_solution = cobra.flux_analysis.pfba(enz_model)
    print("ecGEM max glucose uptake rates:"+str(-enz_model_solution.fluxes[substrate]))
    print("ecGEM max growth rates:"+str(enz_model_solution.fluxes[obj]))
    enz_model_sub=-enz_model_solution.fluxes[substrate]
    enz_model_obj_max=enz_model_solution.fluxes[obj]

    for eachsubid in ori_substrate_id_list:
        if re.search('_reverse',eachsubid):
            r_id_new=eachsubid.split('_reverse')[0]
            enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
            enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0)  
        else:
            r_id_new=eachsubid+'_reverse'
            enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
            enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0) 
    enz_model.reactions.get_by_id(use_substrate).bounds = (-5.05, 0)
    enz_model.reactions.get_by_id(use_substrate+'_reverse').bounds = (0, 0)
    enz_model_obj=enz_model.slim_optimize()

    print('Round %s: '%round_num+str(enz_model_obj))
    round_num=round_num+1
    if round_num>20:
        break
    print()
reaction_kcat_mw.to_csv(reaction_kcat_MW_outfile)

Need changing reaction: 
PPNDH
Changed reaction: 
['PPNDH']
0.129
Enzyme cost total is:0.12900000000000003
Enzyme cost total is:0.12900000000000006
ecGEM max glucose uptake rates:6.332601417282035
ecGEM max growth rates:0.33415990991644673
Round 1: 0.33188367396417223

Need changing reaction: 
ATPS4rpp
Changed reaction: 
['PPNDH']
0.129
Enzyme cost total is:0.12900000000000003
Enzyme cost total is:0.12900000000000006
ecGEM max glucose uptake rates:6.332601417282035
ecGEM max growth rates:0.33415990991644673
Round 2: 0.33188367396417223

Need changing reaction: 
CYTBO3_4pp
Changed reaction: 
['PPNDH']
0.129
Enzyme cost total is:0.12900000000000003
Enzyme cost total is:0.12900000000000006
ecGEM max glucose uptake rates:6.332601417282035
ecGEM max growth rates:0.33415990991644673
Round 3: 0.33188367396417223

Need changing reaction: 
CHORS
Changed reaction: 
['PPNDH', 'CHORS']
0.129
Enzyme cost total is:0.12900000000000003
Enzyme cost total is:0.12900000000000003
ecGEM max glucose uptake 

# Modify PDH kcat by pathway

In [7]:
#change PDH's kcat
reaction_kcat_mw = pd.read_csv(reaction_kcat_MW_outfile, index_col=0)
reaction_kcat_mw.loc['PDH','kcat']=486
reaction_kcat_mw.loc['PDH','kcat_MW']=486* 3600*1000/reaction_kcat_mw.loc['PDH', 'MW']
reaction_kcat_mw.to_csv(reaction_kcat_MW_final_file)

trans_model2enz_json_model_split_isoenzyme(sbml_path, reaction_kcat_MW_final_file, f, ptot, sigma, lowerbound, upperbound, ecModel_final_output_file)
enz_model=get_enzyme_constraint_model(ecModel_final_output_file)

enz_model.objective=obj

#Using only the provided substrate
[ori_obj_id,ori_substrate_id_list,ori_sub_concentration,ori_ATPM]=get_model_substrate_obj(enz_model)
for eachsubid in ori_substrate_id_list:
    if re.search('_reverse',eachsubid):
        r_id_new=eachsubid.split('_reverse')[0]
        enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
        enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0)  
    else:
        r_id_new=eachsubid+'_reverse'
        enz_model.reactions.get_by_id(eachsubid).bounds = (0, 0) 
        enz_model.reactions.get_by_id(r_id_new).bounds = (0, 0) 
        
enz_model.reactions.get_by_id(use_substrate).bounds = (-5.05, 0)
enz_model.reactions.get_by_id(use_substrate+'_reverse').bounds = (0, 0)

enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_final_outfile,ecModel_final_output_file)
print(enz_model_pfba_solution.fluxes[obj])

0.129
Enzyme cost total is:0.129
0.45418779503617907
